In [10]:
import gensim
import gensim.downloader
from sentence_transformers import SentenceTransformer
import pickle
import json
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import os
import itertools
regex_punct = r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"

In [5]:
with open('../Project 1/lr_question_prediction_model/lr.sav', 'rb') as f:
    lr = pickle.load(f)
    
with open('../Project 1/total_labels/color_label2ans.json', 'r') as file:
    total_color_labels = json.load(file)
    
with open('../Project 1/total_labels/number_label2ans.json', 'r') as file:
    total_number_labels = json.load(file)
    
with open('../Project 1/total_labels/length_label2ans.json', 'r') as file:
    total_length_labels = json.load(file)
    
with open('../Project 1/total_labels/time_label2ans.json', 'r') as file:
    total_time_labels = json.load(file)
    
with open('../Project 1/total_labels/bool_label2ans.json', 'r') as file:
    total_bool_labels = json.load(file)
    
with open("../Project 1/VQA_training_code/cluster_data/Using Entire Model/profitable_clusters/profitable_clusters_minDiff_2.pickle", 'rb') as f:
    pc = pickle.load(f)
    
nlp = spacy.load("en_core_web_trf")
st = SentenceTransformer('all-MiniLM-L6-v2')

word2vec = gensim.downloader.load('word2vec-google-news-300')

In [6]:
question_words = ['what', 'which', 'who', 'where', 'why', 'when', 'whose', 'whom', 'how']

dobj_noun = [('dobj', 'NN'), ('dobj', 'NNS')]
nsubj_noun = [('nsubj', 'NN'), ('nsubj', 'NNS')]
nsubjpass_noun = [('nsubjpass', 'NN'), ('nsubjpass', 'NNS')]
pobj_noun = [('pobj', 'NN'), ('pobj', 'NNS')]
simple_noun = ['NN', 'NNS']


def something(list_of_tokens, token_head_text):
    
    if list_of_tokens != []:
        
        for token in list_of_tokens:
            if token.dep_ == 'pobj' and token.tag_ in ['NN', 'NNS']:
                return True, str(token)
            
        children_of_children = list(itertools.chain.from_iterable([list(token.children) for token in list_of_tokens]))
        bool_state, token = something(children_of_children, token_head_text)
        
        return bool_state, str(token)
    
    else:
        return True, token_head_text[0]

def DeterminerCase(spacy_sentence):

    token_head_text = []
    
    for token in spacy_sentence:
        if token.text in question_words and token.dep_ == 'det' and token.head.tag_ in ['NN', 'NNS']:
            children_of_head_token = [child for child in token.head.children if child != token]
            
            token_head_text.append(token.head.text)

   
    if token_head_text != []:
            
        return something(children_of_head_token, token_head_text)
    
    else:
        
        return False, 'No Determiner Case'


def Noun(text):
    
    spacy_sentence = nlp(text)
    
    token_in_string_format = [str(token) for token in spacy_sentence]
    
    dependency = [token.dep_ for token in spacy_sentence]
    pos = [token.tag_ for token in spacy_sentence]
    
    dep_and_pos_zipped = list(zip(dependency, pos))
    
    if DeterminerCase(spacy_sentence)[0] == True:
        
        return [DeterminerCase(spacy_sentence)[1]]
    
    
    elif any(i in dep_and_pos_zipped for i in dobj_noun):
        
        texts = []
        for j in dobj_noun:
            if j in dep_and_pos_zipped:
                indices = [ind for ind, ele in enumerate(dep_and_pos_zipped) if j == ele]
                for index in indices:
                    text = token_in_string_format[index]
                    texts.append(text)
                
        return texts
    

    elif any(i in dep_and_pos_zipped for i in nsubj_noun):
        
        texts = []
        for j in nsubj_noun:
            if j in dep_and_pos_zipped:
                indices = [ind for ind, ele in enumerate(dep_and_pos_zipped) if j == ele]
                for index in indices:
                    text = token_in_string_format[index]
                    texts.append(text)
                
        return texts
    
    
    elif any(i in dep_and_pos_zipped for i in nsubjpass_noun):
        
        texts = []
        for j in nsubjpass_noun:
            if j in dep_and_pos_zipped:
                indices = [ind for ind, ele in enumerate(dep_and_pos_zipped) if j == ele]
                for index in indices:
                    text = token_in_string_format[index]
                    texts.append(text)
                
        return texts


    elif any(i in dep_and_pos_zipped for i in pobj_noun):
        
        texts = []
        for j in pobj_noun:
            if j in dep_and_pos_zipped:
                indices = [ind for ind, ele in enumerate(dep_and_pos_zipped) if j == ele]
                for index in indices:
                    text = token_in_string_format[index]
                    texts.append(text)
                
        return texts
    
    
    elif any(i in pos for i in simple_noun):
        
        texts = []
        for j in simple_noun:
            if j in pos:
                indices = [ind for ind, ele in enumerate(pos) if j == ele]
                for index in indices:
                    text = token_in_string_format[index]
                    texts.append(text)
                
        return texts
    
    else:
        
        return '-'



In [7]:
def Vector(word):
    
    vector = []
    
    word_in_list = word.split(" ")
    for i in word_in_list:
        try:
            vec = word2vec[i]
            vector.append(vec)
        except KeyError:
            pass
        
    if vector != []:
        return sum(vector)
    else:
        return 0
    
    
def MaxCosineValue(word, list_of_words):
    
    word_vec = Vector(word)
    list_of_words_vec = list(map(lambda x : Vector(x), list_of_words))
    list_of_words_vec = list(filter(lambda x: np.all(x!=0), list_of_words_vec))
    max_cosine_value = max(list(map(lambda x: cosine_similarity([x], [word_vec]).tolist()[0][0], list_of_words_vec)))
    
    return max_cosine_value
                        
        

In [8]:
def ClusterSelection(question, answers, pc):
    
    quesAndAns_2_cluster = {}
    
#     print(f"question: {question}, anwsers: {answers}")
    
    nouns = Noun(question)
#     print(f"nouns in question: {nouns}")
        
    for noun in nouns:
        
#         print(f"noun in consideration: {noun}")

        noun_in_clusters = 0

        for cn, cl in pc.items():
            if noun in cn:
                noun_in_clusters += 1
                quesAndAns_2_cluster[(question, answers)] = quesAndAns_2_cluster.get((question, answers), []) + [cl]
                
#                 print(f"'{noun}' found a cluster")
            
        if noun_in_clusters == 0:

#             print(f"'{noun}' did not find any cluster")

            maxCosineValue_and_clusteredLabels_list = []
            for clustered_nouns, clustered_labels in pc.items():
                max_cosine_value = MaxCosineValue(noun, clustered_nouns)
                if max_cosine_value >= 0.4:
                    maxCosineValue_and_clusteredLabels_list.append((max_cosine_value, clustered_labels))

            if maxCosineValue_and_clusteredLabels_list != []:
                
#                 print(f"'{noun}' found a cluster using cosine similarity")
                
                cosine_values_list, clustered_labels_list = zip(*maxCosineValue_and_clusteredLabels_list)
                max_value = max(cosine_values_list)
                max_cosine_value_indices = np.where(np.array(cosine_values_list) == max_value)[0].tolist()
                
                for i in max_cosine_value_indices:
                    quesAndAns_2_cluster[(question, answers)] = quesAndAns_2_cluster.get((question, answers), []) + [clustered_labels_list[i]]
               

    
    return quesAndAns_2_cluster

In [35]:
files = []

for root, dirs, filenames in os.walk("/home/tejan/questions/"):
    files.extend(filenames)


for i, f in enumerate(files[8:10]):
    
    questions = pickle.load(open(f"/home/tejan/questions/{f}", "rb"))

    questions_new = []
    for q in questions.keys():
        q = q.lower()
        ques = re.sub(regex_punct, "", q, 0, re.MULTILINE)
        questions_new.append(ques)

    question_embeddings = st.encode(questions_new, batch_size=64, show_progress_bar=False, convert_to_numpy=True)
    pred = lr.predict(question_embeddings).tolist()
    print(pred)
    
    questions_after_filtering = {}
    for pred, (question, answers) in zip(pred, list(questions.items())):
    
        if pred == 0:
            
            for ans in answers:
                if ans in total_color_labels:
                    questions_after_filtering[question] = {ans}

        elif pred == 1:
            
            for ans in answers:
                if ans in total_number_labels:
                    questions_after_filtering[question] = {ans}


        elif pred == 2:
            
            for ans in answers:
                if ans in total_length_labels:
                    questions_after_filtering[question] = {ans}


        elif pred == 3:
            
            for ans in answers:
                if ans in total_time_labels:
                    questions_after_filtering[question] = {ans}


        elif pred == 4:
            
            for ans in answers:
                if ans in total_bool_labels:
                    questions_after_filtering[question] = {ans}

    
        if pred == 5:
        
            answers = tuple(answers)

            quesAndAns_2_cluster = ClusterSelection(question, answers, pc)
#             print(len(quesAndAns_2_cluster.values()))
            clustered_labels = list(set(itertools.chain.from_iterable(list(quesAndAns_2_cluster.values())[0])))
            for ques, ans in quesAndAns_2_cluster.keys():
                if ans[0] in clustered_labels:
                    print(ans)
                    questions_after_filtering[ques] = set(ans)

    print(questions_after_filtering)

    pickle.dump(questions_after_filtering, open(f"/home/tejan/questions_after_filtering/{f}", "wb"))

[5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
('man',)
('room',)
('couch',)
('backpack',)
('book',)
('chair',)
('man',)
('room',)
('laptop',)
('couch',)
('chair',)
('table',)
('man',)
('lot',)
('man',)
('tv',)
{'Who is standing in the living room with a lot of toys?': {'man'}, 'Where is the man standing?': {'room'}, 'How many toys does the man in the living room have?': {'lot'}, 'What type of furniture has a backpack on it?': {'couch'}, 'What is on the arafed couch with a book on it?': {'backpack'}, 'What is on the back of the arafed couch?': {'book'}, 'A close up of a couch and what other object in the room?': {'chair'}, 'What is the name of the person in the room with a laptop?': {'man'}, 'Where is the laptop?': {'room'}, 'What is he holding in a room?': {'laptop'}, 'Along with a chair, what else is in the room?': {'couch'}, 'Along with a couch, what else is in the room?': {'chair'}, 'Along with a couch and chair, what else is in the room?': {'table'}, 'Who is standing in